<a href="https://colab.research.google.com/github/JHyunjun/TF2.0_2D-CNN/blob/main/2D_CNN_Missingdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

xy_train = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/LSTM/train_v.txt',delimiter=',',dtype=np.float32) #5808
xy_test = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/LSTM/test_v.txt',delimiter=',',dtype=np.float32) #10883
xy_train = np.delete(xy_train ,3, axis = 1)
xy_test = np.delete(xy_test ,3, axis = 1)

start_test = 1000
middle_test = 2000

middle_test1 = 3000
end_test = 10000

xy_test_1 = xy_test[:start_test,:]
xy_test_1n2 = xy_test[start_test:middle_test,:]
xy_test_2 = xy_test[middle_test1:end_test,:]

xy_train = np.concatenate((xy_train, xy_test_1, xy_test_2), axis = 0)
xy_train_Y = np.copy(xy_train)
#xy_test = np.copy(xy_train) # For Adding AWGN
xy_test = xy_test_1n2
xy_test_Y = np.copy(xy_test)

#Normalization

def MinMaxScaler(data) : # Normal Std
  numerator = data - np.min(data,0)
  denominator = np.max(data,0) - np.min(data,0)

  return numerator / (denominator + 1e-7)

def origin_minmax(data) : # HJ STYLE
  arr_max = np.zeros(data.shape[1])
  arr_min = np.zeros(data.shape[1])

  for i in range(data.shape[1]) :
    max = np.max(data[:,i])
    min = np.min(data[:,i])
    arr_max[i] = max
    arr_min[i] = min
    pass
  return arr_max, arr_min

arr_max_train, arr_min_train = origin_minmax(xy_train)
arr_max_test, arr_min_test = origin_minmax(xy_test)
arr_max_total = arr_max_train
arr_min_total = arr_min_train
print(arr_max_train, arr_min_train)
print(arr_max_test, arr_min_test)

for i in range(xy_train.shape[1]) :
  if(arr_max_train[i] > arr_max_test[i]) :
    arr_max_total[i] = arr_max_train[i]
  else :
    arr_max_total[i] = arr_max_test[i]
  pass

for i in range(xy_train.shape[1]) :
  if(arr_min_train[i] < arr_min_test[i]) :
    arr_min_total[i] = arr_min_train[i]
  else :
    arr_min_total[i] = arr_min_test[i]
  pass

print(arr_max_total, arr_min_total)

def HJ_MinMaxScaler(data) :
  for i in range(data.shape[1]) :
    numerator = data[:,i] - arr_min_total[i]
    denominator = arr_max_total[i] - arr_min_total[i]
    data[:,i] = numerator / (denominator + 1e-7)

  return data

def HJ_backMinMax(data) :
  for i in range(data.shape[1]) :
    data[:,i] = data[:,i] * (arr_max_total[i] - arr_min_total[i]) + arr_min_total[i]

  return data

xy_train_scale = HJ_MinMaxScaler(xy_train)
xy_test_scale = HJ_MinMaxScaler(xy_test)
xy_train_Y_scale = HJ_MinMaxScaler(xy_train_Y)
xy_test_Y_scale = HJ_MinMaxScaler(xy_test_Y)

def reshape_timeseries_to_image(data, window_size):
    num_features = data.shape[1]
    num_samples = data.shape[0] - window_size + 1
    reshaped_data = np.zeros((num_samples, window_size, num_features))

    for i in range(num_samples):
        reshaped_data[i] = data[i:i+window_size]

    return reshaped_data

TIME_STEPS = 4

train_image = reshape_timeseries_to_image(xy_train_scale, TIME_STEPS)
test_image = reshape_timeseries_to_image(xy_test_scale, TIME_STEPS)
train_Y_image = reshape_timeseries_to_image(xy_train_Y_scale, TIME_STEPS)
test_Y_image = reshape_timeseries_to_image(xy_test_Y, TIME_STEPS)
print("Train image with Lossing .shape : ", train_image.shape,train_image[0,:,:])
print("Train image.shape : ", train_Y_image.shape,train_image[0,:,:])
print("Test image with Loosing .shape : " ,test_image.shape, test_image[0,:,:])
print("Test image.shape : " ,test_Y_image.shape, test_image[0,:,:])

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.models import Model
from sklearn.metrics import mean_absolute_error

# 이 부분에 실제 데이터 로드 코드를 추가하세요.
# 예시: train_image = np.load('train_image.npy')
# 예시: test_image = np.load('test_image.npy')

# 원본 데이터 복사본 저장
original_data = train_image.copy()

# 각 시퀀스에 대해 N개의 결측치를 만듭니다.
N = 2  # 한 시퀀스 내에서 0으로 만들 값의 수
for i in range(train_image.shape[0]):
    idx_missing = np.random.choice(train_image.shape[1]*train_image.shape[2], N, replace=False)
    train_image[i, idx_missing // train_image.shape[2], idx_missing % train_image.shape[2]] = 0

def data_generator(data, batch_size=32):
    while True:
        # Shuffle data
        np.random.shuffle(data)

        for i in range(0, len(data), batch_size):
            data_batch = data[i : i + batch_size]

            # 데이터 복사본 생성
            data_with_missing = data_batch.copy()
            data_interpolated = data_batch.copy()

            # 결측치 생성 및 보간
            for i in range(data_with_missing.shape[0]):
                idx_missing = np.random.choice(data_with_missing.shape[1]*data_with_missing.shape[2], N, replace=False)
                data_with_missing[i, idx_missing // data_with_missing.shape[2], idx_missing % data_with_missing.shape[2]] = 0
                data_interpolated[i, idx_missing // data_with_missing.shape[2], idx_missing % data_with_missing.shape[2]] = np.mean(data_with_missing[i])

            yield data_interpolated.reshape(data_with_missing.shape[0], data_with_missing.shape[1], data_with_missing.shape[2], 1), data_with_missing

# 자동 혼합 정밀도 설정
policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

train_gen = data_generator(train_image)

# 2D CNN 모델 정의
inputs = Input(shape=(4, 4, 1))
x = Conv2D(256, (3, 3), activation='relu', padding='same')(inputs)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(256, (2, 2), activation='relu', padding='same')(x)
x = Conv2D(256, (2, 2), activation='relu', padding='same')(x)
x = Conv2D(256, (2, 2), activation='relu', padding='same')(x)
outputs = Conv2D(1, (2, 2), activation='linear', padding='same')(x)
outputs = tf.keras.layers.Activation('linear', dtype='float32')(outputs)  # 마지막 레이어는 float32로 설정

model = Model(inputs=inputs, outputs=outputs)

# 모델 컴파일 및 훈련
model.compile(optimizer='adam', loss='mse')
model.fit(train_gen, steps_per_epoch=len(train_image) // 32, epochs=100)

# 결측치 예측
predicted = model.predict(test_image.reshape(test_image.shape[0], test_image.shape[1], test_image.shape[2], 1))

# 0값을 예측값으로 채우기
data_filled = test_image.copy()
zero_indices = np.argwhere(test_image == 0)
for ind in zero_indices:
    data_filled[ind[0], ind[1], ind[2]] = predicted[ind[0], ind[1], ind[2]]

# 복구 정확도 계산
missing_indices = np.argwhere(test_image == 0)
mae = mean_absolute_error(original_data[missing_indices[:, 0], missing_indices[:, 1], missing_indices[:, 2]], data_filled[missing_indices[:, 0], missing_indices[:, 1], missing_indices[:, 2]])
print("복구 정확도 (MAE): ", mae)


In [ ]:
import matplotlib.pyplot as plt

# 예측하기 위해 결측치가 있는 데이터 준비
idx = 0  # 시각화하려는 데이터 인덱스
data_with_missing = test_image[idx].copy()
idx_missing = np.random.choice(data_with_missing.shape[0]*data_with_missing.shape[1], N, replace=False)
data_with_missing[idx_missing // data_with_missing.shape[1], idx_missing % data_with_missing.shape[1]] = 0

# 결측치 예측
predicted = model.predict(data_with_missing.reshape(1, data_with_missing.shape[0], data_with_missing.shape[1], 1))

# 예측값으로 0 채우기
data_filled = data_with_missing.copy()
data_filled[data_with_missing == 0] = predicted[0, data_with_missing == 0]

# 시각화
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

# 원본 데이터 시각화
ax[0].imshow(test_image[idx], cmap='gray')
ax[0].set_title('Original Data')

# 결측치가 있는 데이터 시각화
ax[1].imshow(data_with_missing, cmap='gray')
ax[1].set_title('Data with Missing Values')

# 복구된 데이터 시각화
ax[2].imshow(data_filled, cmap='gray')
ax[2].set_title('Recovered Data')

plt.show()
